# NER as QA for ACE2005

На основе статьи `A Unified MRC Framework for Named Entity Recognition` (https://arxiv.org/abs/1910.11476) и их реализации `mrc-for-flat-nested-ner` (https://github.com/ShannonAI/mrc-for-flat-nested-ner), но при помощи `PyTorch` (https://pytorch.org).

Данная версия Jupyter Notebook для данных `Ace2005` с использованием `BertLarge`. 

### Загрузка данных для ознакомления

Загружаем данные `Ace2005`:

In [1]:
import json

In [2]:
all_data = json.load(open('datasets/ace2005/mrc-ner.train', encoding="utf-8"))

Смотрим на них:

In [3]:
for data in all_data:
    if data['start_position'] != []:
        print(data)
        break

{'context': "BEGALA Well , we ' ll debate that later on in the show .", 'end_position': [3], 'entity_label': 'ORG', 'impossible': False, 'qas_id': '0.2', 'query': 'organization entities are limited to companies, corporations, agencies, institutions and other groups of people.', 'span_position': ['3;3'], 'start_position': [3]}


In [19]:
all_data[13]

{'context': "We ' ll have a couple of experts come out , so I ' ll withhold my comments until then .",
 'end_position': [],
 'entity_label': 'WEA',
 'impossible': True,
 'qas_id': '1.7',
 'query': 'weapon entities are limited to physical devices such as instruments for physically harming such as guns, arms and gunpowder.',
 'span_position': [],
 'start_position': []}

### Построение модели

Подключаем библиотеки:

In [7]:
import argparse
import os
import json
from collections import namedtuple
from typing import Dict

import torch
from tqdm import tqdm
import torch.nn as nn
from models.classifier import MultiNonLinearClassifier, SingleLinearClassifier
from transformers import BertModel, BertPreTrainedModel, BertTokenizer
from tokenizers import BertWordPieceTokenizer
from torch import Tensor
from torch.nn.modules import CrossEntropyLoss, BCEWithLogitsLoss
from torch.utils.data import DataLoader
from transformers import AdamW
from torch.optim import SGD
from metrics.functional.query_span_f1 import query_span_f1

from datasets.mrc_ner_dataset import MRCNERDataset
from datasets.truncate_dataset import TruncateDataset
from datasets.collate_functions import collate_to_max_length
from models.bert_query_ner import BertQueryNER
from models.query_ner_config import BertQueryNerConfig
from loss import *
from utils.get_parser import get_parser
from utils.radom_seed import set_random_seed
import logging

set_random_seed(0)

Строим саму модель, в данном случае в качестве feature extractor берется `BertLarge` (может браться другой), задаются три головы сети - `начальная позиция`, `конечная позиция`, `спан` -- которые будут предсказываться.

In [ ]:
class BertQueryNER(nn.Module):
    def __init__(self):
        super(BertQueryNER, self).__init__()
        self.bert = BertModel.from_pretrained('bert_large/')

        self.start_outputs = nn.Linear(self.bert.config.hidden_size, 1)
        self.end_outputs = nn.Linear(self.bert.config.hidden_size, 1)
        self.span_embedding = MultiNonLinearClassifier(self.bert.config.hidden_size * 2, 1, 0.3)

        self.hidden_size = self.bert.config.hidden_size
        


    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        """
        Args:
            input_ids: bert input tokens, tensor of shape [seq_len]
            token_type_ids: 0 for query, 1 for context, tensor of shape [seq_len]
            attention_mask: attention mask, tensor of shape [seq_len]
        Returns:
            start_logits: start/non-start probs of shape [seq_len]
            end_logits: end/non-end probs of shape [seq_len]
            match_logits: start-end-match probs of shape [seq_len, 1]
        """
        bert_outputs = self.bert(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)

        sequence_heatmap = bert_outputs[0]  # [batch, seq_len, hidden]
        batch_size, seq_len, hid_size = sequence_heatmap.size()

        start_logits = self.start_outputs(sequence_heatmap).squeeze(-1)  # [batch, seq_len, 1]
        end_logits = self.end_outputs(sequence_heatmap).squeeze(-1)  # [batch, seq_len, 1]

        # for every position $i$ in sequence, should concate $j$ to
        # predict if $i$ and $j$ are start_pos and end_pos for an entity.
        # [batch, seq_len, seq_len, hidden]
        start_extend = sequence_heatmap.unsqueeze(2).expand(-1, -1, seq_len, -1)
        # [batch, seq_len, seq_len, hidden]
        end_extend = sequence_heatmap.unsqueeze(1).expand(-1, seq_len, -1, -1)
        # [batch, seq_len, seq_len, hidden*2]
        span_matrix = torch.cat([start_extend, end_extend], 3)
        # [batch, seq_len, seq_len]
        span_logits = self.span_embedding(span_matrix).squeeze(-1)

        return start_logits, end_logits, span_logits

Функция `dataloader`'а, используем `tokenizer_large` (можно другой):

In [3]:
def get_dataloader(data_dir, prefix="train", limit: int = None) -> DataLoader:
    """get training dataloader"""
    """
    load_mmap_dataset
    """
    json_path = os.path.join(data_dir, f"mrc-ner.{prefix}")
    dataset = MRCNERDataset(json_path=json_path,
                            tokenizer= BertWordPieceTokenizer(BertTokenizer.from_pretrained('./tokenizer_large').vocab),
                            max_length=128,
                            is_chinese=False,
                            pad_to_maxlen=False
                            )

    if limit is not None:
        dataset = TruncateDataset(dataset, limit)

    dataloader = DataLoader(
        dataset=dataset,
        batch_size=16,
        num_workers=16,
        shuffle=True if prefix == "train" else False,
        collate_fn=collate_to_max_length
    )

    return dataloader

Загружаем данные уже через `dataloader`:

In [4]:
data_dir = 'ace2005/'
trainloader = get_dataloader(data_dir, 'train')
devloader = get_dataloader(data_dir, 'dev')
testloader = get_dataloader(data_dir, 'test')

/home/easemerova/.conda/envs/my_py/lib/python3.8/site-packages/torch/utils/data/dataloader.py:474: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Задаем веса, на основе статьи вес для спана берется в уменьшенном масштабе:

In [5]:
weight_start = 1
weight_end = 1
weight_span = 0.1
weight_sum = weight_start + weight_end + weight_span
weight_start = weight_start / weight_sum
weight_end = weight_end / weight_sum
weight_span = weight_span / weight_sum
print(weight_start, weight_end, weight_span)

0.47619047619047616 0.47619047619047616 0.047619047619047616


Задаем модель и функцию потерь:

In [6]:
model = BertQueryNER().cuda()
bce_loss = BCEWithLogitsLoss(reduction="none")

Задаем оптимизатор:

In [7]:
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.01,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters,
                  betas=(0.9, 0.98),  # according to RoBERTa paper
                  lr=3e-5)
t_total = (len(trainloader)) * 20
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=3e-5, pct_start=float(0/t_total),
    final_div_factor=1e4,
    total_steps=t_total, anneal_strategy='linear'
)

Функция для обучения:

In [8]:
def train(model, dataloder):
    model.train()
    mean_loss = 0
    for k, batch in enumerate(tqdm(dataloder)):
        tokens, token_type_ids, start_labels, end_labels, start_label_mask, end_label_mask, match_labels, sample_idx, label_idx = batch
        attention_mask = (tokens != 0).long()
        tokens, attention_mask, token_type_ids = tokens.cuda(), attention_mask.cuda(), token_type_ids.cuda()
        start_logits, end_logits, span_logits = model(tokens, attention_mask, token_type_ids)
        start_logits, end_logits, span_logits = start_logits.cpu(), end_logits.cpu(), span_logits.cpu()
        batch_size, seq_len = start_logits.size()

        start_float_label_mask = start_label_mask.view(-1).float()
        end_float_label_mask = end_label_mask.view(-1).float()
        match_label_row_mask = start_label_mask.bool().unsqueeze(-1).expand(-1, -1, seq_len)
        match_label_col_mask = end_label_mask.bool().unsqueeze(-2).expand(-1, seq_len, -1)
        match_label_mask = match_label_row_mask & match_label_col_mask
        match_label_mask = torch.triu(match_label_mask, 0)  # start should be less equal to end

        start_preds = start_logits > 0
        end_preds = end_logits > 0

        match_candidates = torch.logical_or(
            (start_preds.unsqueeze(-1).expand(-1, -1, seq_len)
              & end_preds.unsqueeze(-2).expand(-1, seq_len, -1)),
            (start_labels.unsqueeze(-1).expand(-1, -1, seq_len)
              & end_labels.unsqueeze(-2).expand(-1, seq_len, -1))
        )
        match_label_mask = match_label_mask & match_candidates
        float_match_label_mask = match_label_mask.view(batch_size, -1).float()

        start_loss = bce_loss(start_logits.view(-1), start_labels.view(-1).float())
        start_loss = (start_loss * start_float_label_mask).sum() / start_float_label_mask.sum()
        end_loss = bce_loss(end_logits.view(-1), end_labels.view(-1).float())
        end_loss = (end_loss * end_float_label_mask).sum() / end_float_label_mask.sum()
        match_loss = bce_loss(span_logits.view(batch_size, -1), match_labels.view(batch_size, -1).float())
        match_loss = match_loss * float_match_label_mask
        match_loss = match_loss.sum() / (float_match_label_mask.sum() + 1e-10)


        total_loss = weight_start * start_loss + weight_end * end_loss + weight_span * match_loss
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        scheduler.step()
        mean_loss += total_loss.item()
        if k % 500 == 0 and k > 0:
            print(mean_loss / 500)
            mean_loss = 0

Функция для валидации с выводом `recall`, `precision` и `f1` для полученных спанов, а также их сохранение для каждой эпохи:

In [9]:
def val(model, dataloder, epoch):
    model.eval()
    output = []
    for batch in tqdm(dataloder):
        tokens, token_type_ids, start_labels, end_labels, start_label_mask, end_label_mask, match_labels, sample_idx, label_idx = batch
        attention_mask = (tokens != 0).long()
        tokens, attention_mask, token_type_ids = tokens.cuda(), attention_mask.cuda(), token_type_ids.cuda()
        start_logits, end_logits, span_logits = model(tokens, attention_mask, token_type_ids)
        start_logits, end_logits, span_logits = start_logits.cpu(), end_logits.cpu(), span_logits.cpu()
        start_preds, end_preds = start_logits > 0, end_logits > 0
        span_f1_stats = query_span_f1(start_preds=start_preds, end_preds=end_preds, match_logits=span_logits,
                                      start_label_mask=start_label_mask, end_label_mask=end_label_mask,
                                      match_labels=match_labels)
        output.append(span_f1_stats)
    all_counts = torch.stack(output).sum(0)
    span_tp, span_fp, span_fn = all_counts
    span_recall = span_tp / (span_tp + span_fn + 1e-10)
    span_precision = span_tp / (span_tp + span_fp + 1e-10)
    span_f1 = span_precision * span_recall * 2 / (span_recall + span_precision + 1e-10)
    print(span_recall, span_precision, span_f1)
    with open('./log.txt', 'a') as f:
        f.write(str(epoch) + '\n')
        f.write(str(span_recall.item()) + ' ' + str(span_precision.item()) + ' ' + str(span_f1.item()) + '\n')

### Обучение

Запускаем обучение с сохранением весов:

In [ ]:
for i in range(12 * len(trainloader)):
    scheduler.step()

In [ ]:
for i in range(20):
    train(model, trainloader)
    torch.save(model.state_dict(), 'weights/BertLargeAce2005/weight' + str(i) + '.pth')
    val(model, testloader, i)

 16%|█▌        | 501/3194 [03:53<19:07,  2.35it/s]

0.12858526457287373


 31%|███▏      | 1001/3194 [07:47<17:01,  2.15it/s]

0.075910789469257


 47%|████▋     | 1501/3194 [11:36<11:44,  2.40it/s]

...

 31%|███▏      | 1001/3194 [07:49<18:11,  2.01it/s]

0.0027949979454024287


 47%|████▋     | 1501/3194 [11:42<13:28,  2.09it/s]

0.0033511064438807806


 52%|█████▏    | 1645/3194 [12:48<11:51,  2.18it/s]